In [1]:
import os 
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

from glob import glob
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
physical_device = tf.config.experimental.list_physical_devices('GPU')
print(f'Num of available GPU: {len(physical_device)}')
tf.config.experimental.set_memory_growth(physical_device[0], True)

path_to_training = r'F:\Django\Acute lymphoblastic leukemia\C-NMC_Leukemia\training_data'
path_to_testing = r'F:\Django\Acute lymphoblastic leukemia\C-NMC_Leukemia\testing_data'
path_to_validation = r'F:\Django\Acute lymphoblastic leukemia\C-NMC_Leukemia\validation_data'

Num of available GPU: 1


In [3]:
train_data_gen = ImageDataGenerator(rescale=1/255, 
                                    rotation_range=100,
                                    fill_mode='nearest',
                                    horizontal_flip=True,
                                    vertical_flip=True)
validation_data_gen = ImageDataGenerator(rescale=1/255,
                                  rotation_range=100,
                                  fill_mode='nearest',
                                  horizontal_flip=True,
                                  vertical_flip=True)

In [4]:
train_dataset = train_data_gen.flow_from_directory(path_to_training, 
                                                   target_size=(450, 450),
                                                   batch_size=32,
                                                   class_mode='binary')
validation_dataset = validation_data_gen.flow_from_directory(path_to_validation,
                                                             target_size=(450, 450),
                                                             batch_size=32,
                                                             class_mode='binary')

Found 5088 images belonging to 2 classes.
Found 1690 images belonging to 2 classes.


In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import RMSprop, Adam, SGD
from tensorflow.keras.losses import BinaryCrossentropy

def modelVGG19():
    input_shape = (450, 450, 3)
    input_layer = Input(shape=input_shape)
    
    #Block 1 
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='block1_conv1')(input_layer)
    x = Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same', name='block1_conv2')(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), name='block1_pool')(x)
    
    #Block 2
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='block2_conv1')(x)
    x = Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same', name='block2_conv2')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block2_pool')(x)
    
    #Block 3
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='block3_conv1')(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='block3_conv2')(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='block3_conv3')(x)
    x = Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same', name='block3_conv4')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block3_pool')(x)
    
    #BLock 4
    x = Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='block4_conv1')(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='block4_conv2')(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='block4_conv3')(x)
    x = Conv2D(filters=512, kernel_size=(3, 3), activation='relu', padding='same', name='block4_conv4')(x)
    x = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), name='block4_pool')(x)
    
    #Block 5
    x = Flatten(name='flatten')(x)
    x = Dense(units=128, activation='relu', name='fc1')(x)
    x = Dropout(0.4)(x)
    x = Dense(units=64, activation='relu', name='fc2')(x)
    x = Dropout(0.4)(x)
    output_layer = Dense(units=1, activation='sigmoid')(x)
    
    model = Model(inputs=input_layer, outputs=output_layer)
    model.compile(optimizer=SGD(learning_rate=0.001, decay=1e-6, momentum=0.9, nesterov=True), 
                    loss=BinaryCrossentropy(), 
                    metrics=['accuracy'])
    
    return model

In [6]:
model = modelVGG19()
model.summary()

Model: "functional_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 450, 450, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 450, 450, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 450, 450, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 224, 224, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 224, 224, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 224, 224, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 112, 112, 128)    

In [8]:
try:
    with tf.device('/device:GPU:0'):
        history = model.fit(train_dataset, epochs=32, batch_size=64, validation_data=validation_dataset, verbose=1)
except RuntimeError as e:
    print(e)

Epoch 1/32


ResourceExhaustedError:  OOM when allocating tensor with shape[32,64,450,450] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node functional_1/block1_conv2/Relu (defined at C:\Users\Athenda\AppData\Local\Temp\ipykernel_11212\2194328454.py:3) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_1681]

Function call stack:
train_function
